# 17.1.1 머신러닝(50점)
학생 성적에 관한 데이터에스 제공 394행짜리 소규모 데이터

- 1-4. 데이터 분할 방법을 2가지 쓰고 적절한 데디터 분할을 적용, 선택한 이유 설명
- 1-5. svm, xgboost, randomforest 3개의 알고리즘 공통점을 쓰고, 이 예측 분석에 적합한 알고리즘인지 설명 
- 1-6. 세 가지 모델 모두 모델링 해보고 가장 적합한 알고리즘 선택하고 이유 설명, 한계점 설명하고 보완가능한
부분 설명. 현업에서 사용 시 주의할 점 등에 대해 기술

In [37]:
import pandas as pd
import numpy as np
df = pd.read_csv('./data/student_data_2.csv')
df

,school,sex,paid,activities,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,grade
0,1,0,0,0,4.0,3.0,4.0,1.0,1,3.0,6.0,5,6,6
1,1,0,0,0,5.0,3.0,3.0,1.0,1,3.0,4.0,5,5,6
2,1,0,1,0,4.0,3.0,2.0,2.0,3,3.0,10.0,7,8,10
3,1,0,1,1,3.0,2.0,2.0,1.0,1,5.0,2.0,15,14,15
4,1,0,1,0,4.0,3.0,2.0,1.0,2,5.0,4.0,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0,1,1,0,5.0,5.0,4.0,4.0,5,4.0,11.0,9,9,9
391,0,1,0,0,2.0,4.0,5.0,3.0,4,2.0,3.0,14,16,16
392,0,1,0,0,5.0,5.0,3.0,3.0,3,3.0,3.0,10,8,7
393,0,1,0,0,4.0,4.0,1.0,3.0,4,5.0,0.0,11,12,10


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      395 non-null    int64  
 1   sex         395 non-null    int64  
 2   paid        395 non-null    int64  
 3   activities  395 non-null    int64  
 4   famrel      395 non-null    float64
 5   freetime    395 non-null    float64
 6   goout       395 non-null    float64
 7   Dalc        395 non-null    float64
 8   Walc        395 non-null    int64  
 9   health      395 non-null    float64
 10  absences    395 non-null    float64
 11  G1          395 non-null    int64  
 12  G2          395 non-null    int64  
 13  grade       395 non-null    int64  
dtypes: float64(6), int64(8)
memory usage: 43.3 KB


#### 1-4. 데이터 분할 방법을 2가지 쓰고 적절한 데디터 분할을 적용, 선택한 이유 설명

In [ ]:
데이터를 분할하는 2가지 방법에 '랜덤분할'과 '층화추출기법'이 있는데, 이러한 기법들에 대한 설명은 아래와 같다.
1) 랜덤 분할
  - train/test데이터셋을 나누어서 학습된 데이터를 검증할 수 있다. 
  - 분할 시에 무작위로 사용자가 지정한 비율로 분할한다.
  - 전체 분석 데이터 중 머신러닝 모델을 학습시키기 위한 학습용 데이터와 테스트 데이터로 나누어서 적용시키는 
    이유는 모델 결과가 다른 데이터에도 적용가능한지, 즉 일반화 가능성을 검증하기 위함이다. 
2) 층화추출기법
  - 종속변수의 클래스의 비율이 학습용 데이터와 테스트용 데이터에 비율이 같도록 분할하는 기법이다.
  - 이 기법을 통해 클래스의 편향을 방지할 수 있다. 
  - 이 기법은 종속변수가 범주형일 경우에 사용된다. 
해당 데이터는 종속변수(grade)가 연속형이기 때문에 랜덤샘플링을 사용하였고, train/test비율을 7:3의 비율로
분할하였다.

In [40]:
X = df.drop('grade',axis=1)
y = df['grade']

In [41]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=2022)

#### 1-5. svm, xgboost, randomforest 3개의 알고리즘 공통점을 쓰고, 이 예측 분석에 적합한 알고리즘인지 설명 

In [ ]:
문제에서 제시한 3가지 알고리즘의 공통점은 아래와 같다. 
1. 회귀와 분류 모두 적용할 수 있는 알고리즘이다. 
2. 모두 범주형 변수를 독립변수로 사용할 수 없어서 변환이 필요하다.
3. 과대(과소)적합을 막기위해 하이퍼파라미터 튜닝이 필요하다. 
해당 데이터에서는 종속변수 값이 연속형이므로 회귀분석이 적합하다는 것을 알 수 있다. 

1-6. 세 가지 모델 모두 모델링 해보고 가장 적합한 알고리즘 선택하고 이유 설명, 한계점 설명하고 보완가능한
부분 설명. 현업에서 사용 시 주의할 점 등에 대해 기술

In [8]:
!pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 9.6 MB/s eta 0:00:00


In [42]:
from sklearn.svm import SVR 
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [43]:
# 정규화(SVR 때문)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

#### SVR

In [44]:
# C = 100, gamma = 0.001이 최적임
from sklearn.model_selection import GridSearchCV

param_grid = [
    { 'C': [0.1, 1,10,100],'gamma': [0.001, 0.01, 0.1, 1, 10]}
]

grid_svm = GridSearchCV(SVR(), param_grid =param_grid, cv = 5)
grid_svm.fit(X_train_scaled, y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

,C,gamma,mean_test_score
15,100.0,0.001,0.815206
11,10.0,0.010,0.813794
16,100.0,0.010,0.800047
10,10.0,0.001,0.740380
12,10.0,0.100,0.719862
6,1.0,0.010,0.703161
17,100.0,0.100,0.675613
7,1.0,0.100,0.609533
5,1.0,0.001,0.203629
2,0.1,0.100,0.186046


In [45]:
# 모델링
svr = SVR(C=100, gamma = 0.001) 
svr.fit(X_train_scaled, y_train)
print("R2 : ", svr.score(X_test_scaled, y_test))
print("RMSE:", np.sqrt(mean_squared_error(y_test,svr.predict(X_test_scaled))))

R2 :  0.790510351393822
RMSE: 2.143350008868285


#### 랜덤포레스트

In [53]:
# n_estimators = 100
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators' : [10, 100, 1000]
#     'max_depth' : [5, 10, 15, 20],
#     'min_samples_split' : [2, 5, 10],
#     'min_samples_leaf' : [1, 2, 4]
    }
]
grid_rf = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, cv = 5)
grid_rf.fit(X_train, y_train)

result = pd.DataFrame(grid_rf.cv_results_['params'])
result['mean_test_score'] = grid_rf.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

,n_estimators,mean_test_score
1,100,0.856436
2,1000,0.853413
0,10,0.818680


In [54]:
# 모델링
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train, y_train)
print('R2 : ', rf.score(X_test, y_test))
print('RMSE : ', np.sqrt(mean_squared_error(y_test, rf.predict(X_test))))

R2 :  0.8660475162619952
RMSE :  1.7139067858516215


#### xbgoost

In [55]:
# n_estimators = 100
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators' : [10, 100, 1000]
#     'max_depth' : [5, 10, 15, 20],
#     'learning rate : [0.1, 0.01, 0.001]',
#     'subsample' : [0.8, 0.9, 1.0]  # 트리에 사용할 샘플의 비율
    }
]
grid_xgb = GridSearchCV(XGBRegressor(), param_grid = param_grid, cv = 5)
grid_xgb.fit(X_train, y_train)

result = pd.DataFrame(grid_xgb.cv_results_['params'])
result['mean_test_score'] = grid_xgb.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

,n_estimators,mean_test_score
1,100,0.831101
2,1000,0.831094
0,10,0.831026


In [56]:
# 모델링
xgb = XGBRegressor(n_estimators = 100)
xgb.fit(X_train, y_train)
print('R2 : ', xgb.score(X_test, y_test))
print('RMSE : ', np.sqrt(mean_squared_error(y_test, xgb.predict(X_test))))

R2 :  0.8483109212816207
RMSE :  1.8238493795339665


In [ ]:
가장 적합한 알고리즘은 랜덤포레스트이다. 그 이유는 R2점수값이 가장 높게 나오고, 
에러측면인 RMSE가 가장 낮게 나왔기 때문이다.

부스팅 : train데이터에 과적합될 위험이 있다.
SVR : 차원을 이용하기 때문에 정규화를 시켜줘야 한다.